In [1]:
import pandas as pd #csv파일 읽어오는 모듈
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.sparse.linalg import svds
from ast import literal_eval


import warnings; warnings.simplefilter('ignore')

In [2]:
def return_recipe_id(recipe_title):
    return md[md['recipe_title']==recipe_title]['recipe_id']

def recommend_recipe(df_svd_preds, user_id, ori_recipe_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
   # user_row_number = user_id - 1 
    target_user = df_user.index(user_id)

    # 최종적으로 만든 pred_df에서 사용자 index에 따라 데이터 정렬 -> 레시피 평점(나중엔 스코어) 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[target_user].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_token_id == user_id]
    #user_data.head(10)
    
    # 위에서 뽑은 user_data와 원본 레시피 데이터를 합친다. 
    user_history = user_data.merge(ori_recipe_df, on = 'recipe_id')
    user_history.sort_values("values",axis=0, ascending=False)
    
    # 원본 레시피들에서 사용자가 본 메뉴를 제외한 데이터를 추출
    recommendations = ori_recipe_df[~ori_recipe_df['recipe_id'].isin(user_history['recipe_id'])]
    
    alpha = pd.DataFrame(sorted_user_predictions).reset_index().rename(columns={'index':'recipe_id',target_user:'Predictions'})
    alpha=alpha.astype({'recipe_id':'int'})
    # 사용자의 평점이 높은 순으로 정렬된 데이터와 합친다. 
    recommendations = recommendations.merge(alpha, on = 'recipe_id',how='left')
                                            
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      
    
    return user_history, recommendations#alpha

In [34]:
if __name__ =="__main__":

    recommend_userid =   'test-j60-k20-w20'
    md = pd.read_csv('./yorizori/recipe_yorizori.csv')
    user_info_data = pd.read_csv('./yorizori/yorizori_user.csv').drop(['created_time','updated_time','image_address','nickname','oauth_division'],axis=1)

    target_user_info = user_info_data[user_info_data['user_token_id']==recommend_userid]
    age = target_user_info.iloc[0]["age"]
    gender = target_user_info.iloc[0]["gender"]
    
    if(age=='\\N' or gender=='\\N'):
        df_predict = pd.read_csv('./yorizori_predict/yorizori_predict_matrix.csv')
        user_info = pd.read_csv('./yorizori_predict/yorizori_user_values.csv')
        f = open('./yorizori_predict/yorizori_user_index_info.txt','r')
        df_user = f.readlines()
        for i in range(len(df_user)):
            df_user[i]=df_user[i].split("\n")[0]
            
    else: 
        f = open('./yorizori_predict/yorizori_user_index_info'+age+'_'+gender+'.txt','r')
        df_user = f.readlines()
        for i in range(len(df_user)):
            df_user[i]=df_user[i].split("\n")[0]
            
    
    if(len(df_user)>3):
        df_predict = pd.read_csv('./yorizori_predict/yorizori_predict_matrix'+age+'_'+gender+'.csv')
        user_info = pd.read_csv('./yorizori_predict/yorizori_user_values'+age+'_'+gender+'.csv')
    else :
        df_predict = pd.read_csv('./yorizori_predict/yorizori_predict_matrix.csv')
        user_info = pd.read_csv('./yorizori_predict/yorizori_user_values.csv')
        


    mds=md.drop(['created_time','updated_time','authorship','dish_name','recipe_intro','recipe_thumbnail','user_token_id','reference_recipe','level','time'],axis=1)




    #target_user = df_user.index(recommend_userid)
    already_rated, predictions = recommend_recipe(df_predict, recommend_userid, mds, user_info, 50) 
        #예측행렬,예측하는유저id, 레시피테이블 데이터프레임, 유저별 로그 데이터프레임, 몇개추천할지

    
    print(predictions['recipe_id'].to_list())
    predictions

[1001000, 1001237, 1001029, 1002271, 1001388, 1001034, 1001033, 1001032, 1001030, 1001389, 1001167, 1001391, 1001979, 1000917, 1000915, 1000914, 1001140, 1001168, 1001335, 1001277, 1001775, 1001322, 1001320, 1001282, 1001281, 1001279, 1001262, 1001176, 1001246, 1001816, 1001233, 1001232, 1001221, 1001819, 1000912, 1000908, 1000906, 1002007, 1001750, 1001467, 1001465, 1000070, 1000071, 1002057, 1000780, 1001394, 1002039, 1000805, 1001393, 1001392]


In [35]:

predictions

,recipe_id,recipe_view_count,recipe_title,review_count,star_count,version,Predictions
953,1001000,1932,서리태콩국수,0,0.0,\N,1.232348e-14
1179,1001237,2571,해물샤브샤브,0,0.0,\N,3.774758e-15
980,1001029,1952,모밀국수,0,0.0,\N,3.774758e-15
2127,1002271,518,수정과,0,0.0,\N,3.774758e-15
1324,1001388,401,가리비꼬치구이,0,0.0,\N,3.774758e-15
984,1001034,744,장어소면,0,0.0,\N,3.774758e-15
983,1001033,2203,우동,0,0.0,\N,3.774758e-15
982,1001032,2423,오무라안메밀국수,0,0.0,\N,3.774758e-15
981,1001030,1036,볶음우동,0,0.0,\N,3.774758e-15
1325,1001389,825,갈치노끈소금구이,0,0.0,\N,3.774758e-15


In [10]:
print(age)

\N


In [31]:
predictions

,recipe_id,recipe_view_count,recipe_title,review_count,star_count,version,Predictions
0,1000002,702,고등어초밥,0,0.0,\N,0.0
1849,1001925,2726,해초무침,0,0.0,\N,0.0
1824,1001898,1184,시금치샐러드,0,0.0,\N,0.0
1827,1001901,7182,연어샐러드,0,0.0,\N,0.0
1830,1001904,246,젤리샐러드,0,0.0,\N,0.0
1831,1001905,1593,치즈아보카도,0,0.0,\N,0.0
1832,1001906,1235,치즈토마토,0,0.0,\N,0.0
1833,1001907,1386,치킨텐더,0,0.0,\N,0.0
1835,1001909,2932,컵샐러드,0,0.0,\N,0.0
1836,1001910,2019,케이준치킨샐러드,0,0.0,\N,0.0


In [16]:
age=a.iloc[0]["age"]
gender = a.iloc[0]["gender"]
print(age,gender)

20대 이하 여자
